**Data Collection**

In [3]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [1]:
start_date = '2023-12-1'
end_date = '2024-12-1'
start_date , end_date

('2023-12-1', '2024-12-1')

In [4]:
df = yf.download(tickers='AAPL',start=start_date, end=end_date)
df

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2023-12-01,190.307678,191.240005,191.559998,189.229996,190.330002,45679300
2023-12-04,188.506485,189.429993,190.050003,187.449997,189.979996,43389500
2023-12-05,192.477051,193.419998,194.399994,190.179993,190.210007,66628400
2023-12-06,191.382401,192.320007,194.759995,192.110001,194.449997,41089700
2023-12-07,193.322906,194.270004,195.000000,193.589996,193.630005,47477700
...,...,...,...,...,...,...
2024-11-22,229.869995,229.869995,230.720001,228.059998,228.059998,38168300
2024-11-25,232.869995,232.869995,233.250000,229.740005,231.460007,90152800
